In [ ]:
import time
import random
import iisignature
import numpy as np
from keras import layers
from keras.layers import Conv1D,Conv2D, Input, Lambda, Reshape, Permute, Flatten
from keras.models import Model
from tensorflow import keras  
from keras.optimizers import Adam
from keras.engine import InputSpec
from keras.models import Sequential
from keras.layers import LSTM, Dense, BatchNormalization, Dropout
import tensorflow as tf
from functools import partial
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from cus_layers import *
from PT_logsig_RNN import *

In [ ]:
cha_train = np.load('xsub_train_x_sn.npy')
cha_val = np.load('xsub_test_x_sn.npy')
train_label = np.load('xsub_train_y.npy')
val_label = np.load('xsub_test_y.npy')

# Hyperparameters

In [ ]:
n_hidden_neurons = 256
batch_size = 256
learning_rate = 0.001
epochs = 60
number_of_segment = 32
deg_of_logsig = 2
drop_rate_1 = 0
drop_rate_2 = 0.8
filter_size = 40
output_shape = 120
input_shape = [72, 25,6]
Time = []
Acc = []

# Build model

In [ ]:
model = build_lin_Logsig_rnn_model(input_shape, n_hidden_neurons, output_shape, number_of_segment, deg_of_logsig, learning_rate, drop_rate_1
                                                   ,drop_rate_2, filter_size)

In [ ]:
model_name = 'ntu_xsub_lstm+conv+logsig.hdf5' 
# Keras callbacks for reducing the learning rate and saving the model
# call back to early stop if the loss doesn't decrease for <patience=300> epochs
early_stopping_monitor = EarlyStopping(monitor='loss', min_delta=0, patience=20, verbose=0, mode='auto')
# Call back to reduce learning rate by <factor=0.8> if the loss doesn't decrease for <patience=100> epochs
reduce_lr = ReduceLROnPlateau(monitor='loss', patience=50, verbose=1, factor=0.8, min_lr=0.000001)
# call back to save model ( saves best model only )
mcp_save = ModelCheckpoint(model_name, save_best_only=True, monitor='acc', mode='auto')

# Train model

In [ ]:
hist = model.fit(cha_train.reshape(cha_train.shape[0],72,25,6), train_label, epochs=epochs, batch_size=batch_size,shuffle=True, 
                    verbose=1, callbacks = [ early_stopping_monitor, reduce_lr, mcp_save])

# Test model

In [ ]:
model.evaluate(cha_val.reshape(cha_val.shape[0],72,25,6), val_label)